In [36]:
import pandas as pd
import urllib.request
from datetime import datetime
from datetime import datetime, timedelta 
import geopandas as gpd
import geoplot
import geoplot.crs as gcrs
import matplotlib.pyplot as plt
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import urllib.request
import tarfile
import shapely
from shapely import wkt
from joblib import Parallel, delayed
import pandas as pd


In [37]:
date = datetime.today() + timedelta(days=1) 
date = date.strftime('%Y-%m-%d')

date1 = datetime.today() - timedelta(days=30*365) 
date1 = date1.strftime('%Y-%m-%d')

url = urllib.request.urlretrieve('https://mesonet.agron.iastate.edu/cgi-bin/request/gis/lsr.py?&type=FLOOD&sts='+date1+'T00:00Z&ets='+date+'T00:00Z&fmt=csv', r'report.csv')
df = pd.read_csv('report.csv', on_bad_lines='skip')


In [38]:
#Fill remark column with '' in place of Nans
df['TYPETEXT'] = df['TYPETEXT'].fillna('')

#df = df.dropna()
df = df[df['TYPETEXT'].str.contains("FLOOD")]
df = df[~df['TYPETEXT'].str.contains("FLASH FLOOD")]
df = df[~df['TYPETEXT'].str.contains("COASTAL FLOOD")]
#df = df[df.LON > -130]
#df = df[df.LON < -50]

df['LAT'] = df['LAT'].astype(float)
#df = df[df.LAT > 20.00]


#Change VALID2 time to datetime format
df['VALID2'] = pd.to_datetime(df['VALID2'])

In [39]:
#Needs work, script is not actually filtering the LSRs



#Filter to find most likely river flooding events.
#df =  df[df['REMARK'].str.contains("closed|river|River")]
#df =  df[df['REMARK'].str.contains("River")]
#df.to_csv("River_Only.csv")

import re
# Remove column name 'MAG'
df.drop(['MAG'], axis=1)

#keywords = ["River", "stream", "overbank", "creek", "Creek", "Bayou", "bayou", "Banks", "banks", "Bank", "bank", "gage", "Gage", "road"]
# Function to check if any word in a text is in the list of keywords
# Create a regular expression pattern from the list of keywords
#pattern = '|'.join(keywords)

#Fill remark column with '' in place of Nans
df['REMARK'] = df['REMARK'].fillna('')


# Search for rows containing any of the keywords
#df = df[df['REMARK'].str.contains(pattern, flags=re.IGNORECASE)]

df.to_csv("River_Only.csv")

#df = df.head(20)

In [40]:
#Create geodataframe
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.LON, df.LAT), crs="EPSG:4326")

#Change Projection
gdf = gdf.to_crs(epsg=3857)

In [41]:
df['REMARK']=df['REMARK'].astype(str)


In [42]:
#This will download the latest ahps shapefile

#Download data and save the zipped shapefile
urllib.request.urlretrieve('https://water.noaa.gov/resources/downloads/shapefiles/national_shapefile_fcst_ffep.tgz', r'ahps.tgz')

# For future refence, it appears that the national_shapefile_fcst_ffep.tgz file above has a few more sites than the observed file below.
# Therefore, I opted to use the national_shapefile_fcst_ffep.tgz.

#urllib.request.urlretrieve('https://water.noaa.gov/resources/downloads/shapefiles/national_shapefile_obs.tgz', r'ahps.tgz')
#The file is zipped so now we have to unzip the file like we normally do with an AHPS Shapefile
fname = 'ahps.tgz'
if fname.endswith("ahps.tgz"):
    tar = tarfile.open(fname, "r:gz")
    tar.extractall()
    tar.close()
elif fname.endswith("tar"):
    tar = tarfile.open(fname, "r:")
    tar.extractall()
    tar.close()


In [43]:
#Every shapefile has a dbf file which is a table.We can now read that table and create a Pandas Dataframe with it.  
table = gpd.read_file(r'national_shapefile_fcst_ffep.dbf')
ahps = pd.DataFrame(table)
#This will save it if you want to. It may be good to look at.
ahps.to_csv('ahps.csv')
#View the table just by using this line
#ahps

In [44]:
# Convert the ahps DataFrame to a GeoDataFrame, adding a geometry column from the 'Latitude' and 'Longitude' columns
# We're converting the ahps DataFrame to a GeoDataFrame with point geometries to enable spatial operations

ahps = gpd.GeoDataFrame(ahps, geometry=gpd.points_from_xy(ahps.Longitude, ahps.Latitude), crs="EPSG:4326")

#Change Projection
ahps = ahps.to_crs(epsg=3857)

# Perform a spatial join between the GeoDataFrame gdf and the GeoDataFrame ahps, 
# keeping only the nearest match from ahps for each geometry in gdf.
# This finds the nearest gage (point) for each geometry in gdf
gdf_joined = gpd.sjoin_nearest(gdf, ahps, how='left')

# Create a copy of the resulting GeoDataFrame gdf_joined
# Creating a copy ensures that modifications to gdf won't affect gdf_joined
gdf = gdf_joined.copy()

# Convert the 'Latitude' and 'Longitude' columns to string data type
# This conversion might be necessary for later string operations
gdf = gdf.astype({'Latitude': 'str', 'Longitude': 'str'})

# Create a new column 'geometry_ahps' by concatenating the 'Longitude' and 'Latitude' columns 
# and forming a Well-Known Text (WKT) representation of a point geometry
# This creates a WKT representation of point geometries from the 'Latitude' and 'Longitude' columns
gdf['geometry_ahps'] = "POINT ("+ gdf['Longitude']+" "+gdf['Latitude']+")"

# Convert the 'geometry_ahps' column from WKT representation to Shapely geometries
# Convert WKT strings to Shapely Point geometries
gdf['geometry_ahps'] = gdf['geometry_ahps'].apply(wkt.loads)

In [45]:
# Create a GeoSeries from the 'geometry_ahps' column and set its CRS to EPSG:4326
gdf['geometry_ahps'] = gpd.GeoSeries(gdf['geometry_ahps'], crs="EPSG:4326")

# Ensure both geometry columns have the same CRS
if gdf.crs != gdf['geometry_ahps'].crs:
    gdf['geometry_ahps'] = gdf['geometry_ahps'].to_crs(gdf.crs)

In [46]:
# Calculate the distance between the 'geometry' and 'geometry_ahps' for each row in the GeoDataFrame
# and store the result in a new column named 'distance'
# This calculates the distance between each geometry in gdf and its corresponding geometry in 'geometry_ahps'
gdf["distance"] = gdf.apply(lambda row: row["geometry"].distance(row["geometry_ahps"]), axis=1)

# Convert the distance from meters to miles and store the result in a new column named 'distance_miles'
# Convert distances from meters to miles for convenience
gdf["distance_miles"] = gdf["distance"]/1609

#This will give us all the LSRs within 5 miles of a gage, which might be a good place to start.
gdf = gdf[gdf['distance_miles'] < 5]

gdf.to_csv('delete.csv')

In [47]:

urllib.request.urlretrieve('https://water.noaa.gov/resources/downloads/reports/nwps_all_gauges_report.csv',r'nwps_all_gauges_report.csv')

ahps = pd.read_csv('nwps_all_gauges_report.csv', index_col=False)

ahps = ahps[ahps['usgs id'].notna()]


In [48]:
# Create a new column 'GaugeLID' and set it equal to the values in the 'nws shef id' column
# Convert the values to strings and convert them to uppercase
ahps['GaugeLID'] = ahps['nws shef id']
ahps['GaugeLID'] = ahps['GaugeLID'].astype(str)
ahps['GaugeLID'] = ahps['GaugeLID'].str.upper()

# Keep only the 'GaugeLID' and 'usgs id' columns in the DataFrame
# This is to retain only the necessary information for merging with the main DataFrame gdf
ahps = ahps[['GaugeLID', 'usgs id', 'action stage', 'flood stage','moderate flood stage', 'major flood stage']]

# Merge the main DataFrame gdf with the ahps DataFrame on the 'GaugeLID' column, using a left join
# This adds the USGS ID information to the main DataFrame gdf based on the 'GaugeLID' column
gdf = pd.merge(gdf, ahps, how='left', on="GaugeLID")

In [49]:
# Convert the 'datetime_column' to datetime format for begin date.
gdf['begin_date'] = pd.to_datetime(gdf['VALID2'], format='%Y/%m/%d %H:%M')
gdf['begin_date']=gdf['begin_date']- pd.to_timedelta(1, unit='d')
gdf['begin_date']=gdf['begin_date'].dt.strftime('%Y-%m-%d')

# Convert the 'datetime_column' to datetime format for end date.
gdf['end_date'] = pd.to_datetime(gdf['VALID2'], format='%Y/%m/%d %H:%M')
gdf['end_date']=gdf['end_date']+ pd.to_timedelta(1, unit='d')
gdf['end_date']=gdf['end_date'].dt.strftime('%Y-%m-%d')

In [50]:
gdf.to_csv('delete.csv')

In [51]:

def return_status(row):  
    try:    
        site = row['usgs id']
        begin_date = row['begin_date']
        end_date = row['end_date']

        action = float(row['action stage'])
        minor = float(row['flood stage'])
        moderate = float(row['moderate flood stage'])
        major = float(row['major flood stage'])

        event_time = row['VALID2']

        table = f'https://nwis.waterdata.usgs.gov/usa/nwis/uv/?cb_00065=on&format=rdb&site_no={site}&period=&begin_date={begin_date}&end_date={end_date}'
        print(table)
        df = pd.read_fwf(table, comment='#')
        df = df.iloc[1:,0].str.split("\t", expand=True)
        df.rename(columns={0:'Agency', 1:'USGSstationID', 2:'Date', 4:'Stage'}, inplace=True)
        df['Stage'] = pd.to_numeric(df['Stage'], errors='coerce').dropna()
        df['Date1'] = pd.to_datetime(df['Date'])
        df['date'] = df['Date1'].dt.strftime('%Y-%m-%d, %H'+'00')

        def f(row):
            if row['Stage'] > major and major != -9999:
                return 'major'
            elif row['Stage'] > moderate and moderate != -9999:
                return 'moderate'
            elif row['Stage'] > minor and minor != -9999:
                return 'minor'
            elif row['Stage'] > action and action != -9999:
                return 'action'
            return 'none'

        df['status'] = df.apply(f, axis=1) 

        event_row_index = (df['Date1'] - event_time).abs().idxmin()
        event_row = df.loc[event_row_index]
        return event_row['Stage'], event_row['status'], event_row['date']

    except Exception as e:
        return 'error: USGS', 'error: USGS', 'error: USGS'

# Parallel processing using joblib
num_cores = -1  # Uses all available cores
results = Parallel(n_jobs=num_cores)(delayed(return_status)(row) for _, row in gdf.iterrows())

# Assign results back to the dataframe
gdf[['Event stage (ft)', 'Event status', 'Event date']] = pd.DataFrame(results, index=gdf.index)


In [52]:
gdf.to_csv('river_impacts.csv')


In [53]:




# Read the CSV into a GeoDataFrame
gdf = pd.read_csv('river_impacts.csv')



#Create lsr url for lsr
#Change time to string so it can be used to make the url
gdf['VALID'] = gdf['VALID'].astype(str)

#Create URL
gdf['lsr_url'] = 'https://mesonet.agron.iastate.edu/lsr/#'+gdf['WFO_left']+'/'+gdf['VALID']+'/'+gdf['VALID']+'/010010'

gdf['VALID2'] = pd.to_datetime(gdf['VALID2'], errors='coerce')

# Define a function to create the suggested comment for each row
def create_suggested_comment(row):
    return (
        'There is potential for flooding at this location if the current flow were to materialize. \n\n' +
        f'This is supported by an LSR (link below) that occurred on {row["VALID2"].strftime("%Y-%m-%d")} at {row["VALID2"].strftime("%H:%M")}\n\n' +
        f'The nearby gage ({row["GaugeLID"]}) on the {row["Waterbody"]} reported a stage and status of {row["Event stage (ft)"]} ft and {row["Event status"]} flood.\n\n' +
        row['lsr_url']
    )

# Apply the function to each row and create the 'suggested_comment' column
gdf['suggested_comment'] = gdf.apply(create_suggested_comment, axis=1)

# Convert 'VALDID2' column to string
gdf['VALID2'] = gdf['VALID2'].dt.strftime('%Y-%m-%d')  # Or any format you prefer




In [54]:
#Two columns ended up being State, to fix this the 25th column was called directly to rename specifically
gdf.rename(columns={gdf.columns[25]: "Gage State"}, inplace=True)

#Renaming columns
gdf.rename(columns={
    'VALID': 'LSR Date/Time', 'LAT': 'Lat', 'LON': 'Lon', 'WFO_left': 'WFO', 'TYPETEXT': 'Event Type',
    'CITY': 'City', 'COUNTY': 'County', 'STATE':'State', 'SOURCE': 'Source', 'REMARK': 'Remark',
    'GaugeLID': 'Nearest Gage', 'Location': 'Gage Location', 'Waterbody': 'Gage Waterbody', 
    'Action': 'Action Threshold', 'Flood': 'Minor Threshold', 'Moderate': 'Moderate Threshold',
    'Major': 'Major Threshold', 'URL': 'Gage URL', 'distance_miles': 'LSR to Gage Distance (miles)',
    'usgs id': 'USGS ID', 'Event stage (ft)': 'Event Stage (ft)', 'Event status': 'Event_Status',
    'Event date': 'Gage Observation Date/Time', 'lsr_url': 'LSR Link', 'suggested_comment': 'Suggested Comment'}, inplace=True)

#Deletes any leading/trailing spaces in column names
gdf.columns = gdf.columns.str.strip()
# List of columns to drop
gdf.drop(columns=[j for j in [
    'Unnamed: 0', 'VALID2', 'MAG', 'TYPECODE', 'UGC', 'UGCNAME', 'QUALIFIER', 
    'index_right', 'Status', 'Latitude', 'Longitude', 'Forecast', 'FcstTime', 
    'FcstIssunc', 'Units', 'LowThresh', 'LowThreshU', 'WFO_right', 'HDatum', 
    'PEDTS', 'SecValue', 'SecUnit', 'geometry_ahps', 'distance', 'action stage', 
    'flood stage', 'moderate flood stage', 'major flood stage', 'begin_date', 'end_date'] 
    if j in gdf.columns], inplace=True)

# Define mapping for event status values
status_mapping = {
    'action': 1,
    'minor': 2,
    'moderate': 3,
    'major': 4
}

# Create the new column with mapped values, defaulting to 0 for unmapped values
gdf['Event Status Code'] = gdf['Event_Status'].map(status_mapping).fillna(0).astype(float)

#Filter to only minor and higher
gdf = gdf[gdf['Event Status Code'] > 1]

gdf.to_csv('gdf.csv', index=False)

In [55]:
#Filter to only minor and higher
gdf = gdf[gdf['Event Status Code'] > 1]

gdf['Event Status Code'] = gdf['Event Status Code'].astype(float)

In [56]:
#Set the projection
lsrs = gpd.GeoDataFrame(gdf, geometry=gpd.points_from_xy(gdf.Lon, gdf.Lat), crs="EPSG:4326")

# Reproject to Web Mercator (EPSG:3857)
lsrs = lsrs.to_crs("EPSG:3857")

# Save the GeoDataFrame as a shapefile
lsrs.to_file("lsrs_shapefile.shp")

C:\Users\david.smith\AppData\Local\Temp\ipykernel_12668\1927496774.py:8: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  lsrs.to_file("lsrs_shapefile.shp")


In [57]:

# Save as a GeoPackage
lsrs.to_file("lsrs_feature_layer.gpkg", driver="GPKG")